In [1]:
import pandas as pd
import numpy as np

In [2]:
path = '/home/gas/Documents/Yandex_ML_track_2018/'

In [3]:
train = pd.read_csv(path+'data/train.tsv', sep='\t', quoting=3, header=None)
test = pd.read_csv(path+'data/public.tsv', sep='\t', quoting=3, error_bad_lines=False, header=None)

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97533 entries, 0 to 97532
Data columns (total 8 columns):
0    97533 non-null int64
1    48559 non-null object
2    72502 non-null object
3    97533 non-null object
4    97533 non-null int64
5    97533 non-null object
6    97533 non-null object
7    97533 non-null float64
dtypes: float64(1), int64(2), object(5)
memory usage: 6.0+ MB


In [5]:
train.fillna('', inplace=True)
test.fillna('', inplace=True)

In [6]:
# расставляем значения по формуле хорошесть * confidence
# Good (3)
# Neutral (2)
# Bad (1)

In [7]:
def rank2num(st):
    if st == 'good':
        return 3
    else:
        if st == 'neutral':
            return 2
        else:
            return 1

In [8]:
train['rank'] = train[6].apply(rank2num)
train['target'] = train['rank'] * train[7]
train.head()

,0,1,2,3,4,5,6,7,rank,target
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",0,не могу .,good,0.875352,3,2.626055
1,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",1,"нет , звонить буду я .",neutral,0.900968,2,1.801936
2,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",2,"слушай , я не мог уйти .",bad,0.884320,1,0.884320
3,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",3,я не прекращу звонить .,good,0.982530,3,2.947591
4,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",4,я звоню им .,good,0.838054,3,2.514161


In [9]:
test.head()

,0,1,2,3,4,5
0,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,0,неа .
1,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,1,"нет , не хочу ."
2,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,2,нет .
3,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,3,"конечно , нет ."
4,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,4,"разумеется , нет ."


In [10]:
import nltk
#nltk.download('wordnet')

from collections import OrderedDict
from gensim.utils import deaccent
import operator
import pickle

class getTokens:
    
    def __init__(self, garbage=['hello', 'hi', 'need', 'help', 'com', 'http', 'please', 'someone', 'etc', 'www']):
        self.tokenizer = nltk.RegexpTokenizer('\w+')        
        self.lmtzr = nltk.stem.wordnet.WordNetLemmatizer()
        self.garbage = set(garbage)
    
    def transform(self, text):
        text = self.tokenizer.tokenize(text)
        text = [self.lmtzr.lemmatize(deaccent(word.lower())) for word in text]
        text = [word for word in text if len(word) > 1 and word not in self.garbage]
        return ' '.join(text)

/home/gas/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
tokenize = getTokens()

In [12]:
text_cols = [1, 2, 3, 5]

In [13]:
for col in text_cols:
    train[col] = train[col].apply(tokenize.transform)
    test[col] = test[col].apply(tokenize.transform)

In [20]:
train.head()

,0,1,2,3,4,5,6,7,rank,target
0,22579918886,кликни на меня потом на надпись видео звонок,тебя вижу,ладно повесь трубку,0,не могу,good,0.875352,3,2.626055
1,22579918886,кликни на меня потом на надпись видео звонок,тебя вижу,ладно повесь трубку,1,нет звонить буду,neutral,0.900968,2,1.801936
2,22579918886,кликни на меня потом на надпись видео звонок,тебя вижу,ладно повесь трубку,2,слушаи не мог уити,bad,0.884320,1,0.884320
3,22579918886,кликни на меня потом на надпись видео звонок,тебя вижу,ладно повесь трубку,3,не прекращу звонить,good,0.982530,3,2.947591
4,22579918886,кликни на меня потом на надпись видео звонок,тебя вижу,ладно повесь трубку,4,звоню им,good,0.838054,3,2.514161


In [21]:
test.head()

,0,1,2,3,4,5
0,138920940977,знаешь иногда подумываю что тебе надо принести...,не,нет,0,неа
1,138920940977,знаешь иногда подумываю что тебе надо принести...,не,нет,1,нет не хочу
2,138920940977,знаешь иногда подумываю что тебе надо принести...,не,нет,2,нет
3,138920940977,знаешь иногда подумываю что тебе надо принести...,не,нет,3,конечно нет
4,138920940977,знаешь иногда подумываю что тебе надо принести...,не,нет,4,разумеется нет


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import pipeline

In [15]:
svd_n_components = 300
max_features = 500000

In [262]:
counters_pipe = pipeline.FeatureUnion(
    n_jobs = -1,
    transformer_list = [
    ('chars_features', pipeline.Pipeline([
        ('chars_counter', CountVectorizer(
            analyzer=u'char', ngram_range=(2, 4), tokenizer=None,
            max_features=max_features, strip_accents=None, max_df=0.9, min_df=2, lowercase=False)),
        ('chars_tsvd', TruncatedSVD(n_components=svd_n_components, n_iter=25, random_state=42))])),
    ('words_features', pipeline.Pipeline([
        ('words_counter', CountVectorizer(
            analyzer=u'word', ngram_range=(1, 3), tokenizer=None,
            max_features=max_features, strip_accents=None, max_df=0.9, min_df=2, lowercase=False)),
        ('words_tsvd', TruncatedSVD(n_components=svd_n_components, n_iter=25, random_state=42))])),
])

tfidf_pipe = pipeline.Pipeline([
    ('words_tfidf', TfidfVectorizer(
       analyzer=u'word', ngram_range=(1, 3), tokenizer=None,
       max_features=max_features, strip_accents=None, max_df=0.9, min_df=2, lowercase=False)),
    ('words_tsvd', TruncatedSVD(n_components=svd_n_components*2, n_iter=25, random_state=42))])

In [16]:
from gensim.models import KeyedVectors

skipgram_ru = KeyedVectors.load_word2vec_format(path+'/models/ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz')
lmtzr = nltk.stem.wordnet.WordNetLemmatizer()
new_skipgram_ru = {}
for word in skipgram_ru.vocab.keys():
    new_word = lmtzr.lemmatize(deaccent(word.split('_')[0]))
    new_skipgram_ru[new_word] = skipgram_ru[word]
del skipgram_ru

In [17]:
tfidf = TfidfVectorizer(
       analyzer=u'word', ngram_range=(1, 1), tokenizer=None, use_idf=True,
       max_features=10000, strip_accents=None, max_df=0.9, min_df=2, lowercase=False)

In [ ]:
def weighter(transformed, tfidf, new_skipgram_ru):
    data = None
    for i in range(transformed.shape[0]):
        features = 0
        for j, word in enumerate(tfidf.vocabulary_):
            if word in new_skipgram_ru.keys():
                features += new_skipgram_ru[word] * transformed[i, tfidf.vocabulary_[word]]
        if data is None:
            data = features
        else:
            data = np.vstack([data, features])
    return data

In [98]:
tfidf_data = tfidf.fit_transform(train[1].loc[:100])

/home/gas/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [18]:
w2v = None
for i, word in enumerate(tfidf.vocabulary_.keys()):
    if word not in new_skipgram_ru.keys():
        if w2v is None:
            w2v = np.zeros(300)
        else:
            w2v = np.vstack([w2v, np.zeros(300)])
        continue
    if w2v is None:
        w2v = new_skipgram_ru[word]
    else:
        w2v = np.vstack([w2v, new_skipgram_ru[word]])

/home/gas/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [21]:
new_skipgram_ru = pd.DataFrame.from_dict(new_skipgram_ru, orient='index')

KeyboardInterrupt: 

In [342]:
embeddings.head()

,!ки,#0,#000000,#1,#1-6,#10,#11,#12,#13,#14,...,年號::nianhao,従五位下,従四位下,新編::日本史辞典,日本,東京,男はつらいよ,运城专区,郊区,"�,�"
0,-0.051333,0.125593,0.016969,0.103200,0.089624,0.072397,0.133636,0.095831,0.111811,0.085971,...,-0.028292,0.034690,0.074955,0.074041,0.018757,-0.008806,0.035595,0.047633,0.031999,0.020035
1,0.114001,0.060816,0.105114,0.072320,0.048856,0.072119,0.043670,-0.004487,0.099241,0.032873,...,0.010153,0.054273,0.025466,0.065673,0.083741,0.104142,0.040070,0.089000,0.082389,0.031989
2,-0.086864,0.132568,-0.047530,0.102536,0.060635,-0.014821,0.058024,0.065031,0.053722,0.024626,...,0.017632,0.110340,0.096492,0.100238,0.006842,0.004834,0.056163,0.068382,0.043812,0.011578
3,-0.050628,-0.014923,-0.055164,0.005984,-0.036993,0.008021,-0.002301,0.000520,0.053647,0.065537,...,0.009815,-0.027986,0.037767,0.100304,-0.067201,-0.110351,0.021179,0.069392,-0.025067,-0.039870
4,-0.018447,-0.056804,-0.110744,-0.021028,-0.006750,-0.024681,-0.014100,0.003425,0.053173,0.010043,...,-0.058368,-0.138596,-0.132692,-0.075596,-0.104805,-0.015554,0.018849,-0.046377,-0.065172,-0.068840


In [339]:
tfidf_voc = pd.DataFrame(tfidf.vocabulary_, index=range(len(tfidf.vocabulary_)))

In [22]:
tfidf_data.shape

(101, 61)

In [23]:
w2v.shape

(61, 300)

In [333]:
tfidf_data *= w2v

In [32]:
tfidf_data = tfidf_data.todense()

In [67]:
w2v[idxes].shape, tfidf_data[0, idxes]

((7, 300), matrix([[0.31622777, 0.31622777, 0.31622777, 0.31622777, 0.63245553,
          0.31622777, 0.31622777]]))

In [109]:
new_tfidf_data = np.empty((tfidf_data.shape[0], 300))

In [110]:
new_tfidf_data.shape

(101, 300)

In [103]:
idxes = np.where(tfidf_data[0].todense())[1]
test1 = np.dot(tfidf_data[0, idxes].todense(), w2v[idxes])
#(w2v[idxes]*tfidf_data[0, idxes])

In [106]:
test1.shape

(1, 300)

In [89]:
test2 = w2v[idxes[0]]*tfidf_data[0, idxes[0]]
for id_ in idxes[1:]:
    test2 += w2v[id_]*tfidf_data[0, id_]

In [93]:
test1.all() == test2.all()

True

In [107]:
new_tfidf_data[0, :] = test1

In [108]:
new_tfidf_data.shape

(101, 300)

In [303]:
len(tfidf.vocabulary_)

61

In [304]:
w2v.shape

(61, 300)

In [280]:
tfidf.vocabulary_

{'бывало': 0,
 'бывало получше': 1,
 'видео': 2,
 'видео звонок': 3,
 'звонок': 4,
 'кликни': 5,
 'кликни на': 6,
 'кликни на меня': 7,
 'меня': 8,
 'меня потом': 9,
 'меня потом на': 10,
 'на': 11,
 'на меня': 12,
 'на меня потом': 13,
 'на надпись': 14,
 'на надпись видео': 15,
 'надпись': 16,
 'надпись видео': 17,
 'надпись видео звонок': 18,
 'получше': 19,
 'потом': 20,
 'потом на': 21,
 'потом на надпись': 22}

In [279]:
tfidf_data

<11x1 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [275]:
w2v = weighter(tfidf_data, tfidf, new_skipgram_ru)

In [277]:
w2v.shape

(11, 300)

In [116]:
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.

def ndcg_at_k(r, k=3, method=0):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [53]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold

In [113]:
nfolds = round(len(train) / len(test))

lr = Ridge(alpha=3.0)
cv = KFold(n_splits=nfolds, shuffle=True, random_state=42)

In [114]:
fold = 1
cv_score = 0
for train_index, test_index in cv.split(train):
    print('make features...')
#     counters_pipe.fit(np.hstack([train[col].loc[train_index] for col in text_cols]))
#     train_data = np.hstack([counters_pipe.transform(train[col].loc[train_index]) for col in text_cols])
#     test_data = np.hstack([counters_pipe.transform(train[col].loc[test_index]) for col in text_cols])
    
    tfidf.fit(np.hstack([train[col].loc[train_index] for col in text_cols]))
    train_data = np.hstack([weighter(tfidf.transform(train[col].loc[train_index]), tfidf, new_skipgram_ru) for col in text_cols])
    test_data = np.hstack([weighter(tfidf.transform(train[col].loc[test_index]), tfidf, new_skipgram_ru) for col in text_cols])
    
    print('fitting model...')
    lr.fit(train_data, train['target'].loc[train_index])

    print('make prediction...')
    prediction = pd.DataFrame()
    prediction['context_id'] = train[0].loc[test_index]
    prediction['reply_id'] = train[4].loc[test_index]
    prediction['rank'] = - lr.predict(test_data)
    prediction = prediction.sort_values(by=['context_id', 'rank'])[['context_id', 'reply_id']]
    
    score = 0
    uniques = prediction['context_id'].unique()
    for Id in uniques:
        tmp = prediction[prediction['context_id'] == Id]['reply_id'].values.tolist()
        score += ndcg_at_k(tmp, len(set(tmp))) / len(uniques)
    score *= 100000
    cv_score += score / nfolds
    
    print('fold#%i: ndcg = %i' % (fold, int(score)))
    fold += 1
print('averaged cv score: ndcg = %i' % (int(cv_score)))

make features...
fitting model...
make prediction...
fold#1: ndcg = 85496
make features...
fitting model...
make prediction...
fold#2: ndcg = 85920
make features...
fitting model...
make prediction...
fold#3: ndcg = 85909
make features...
fitting model...
make prediction...
fold#4: ndcg = 86006
make features...
fitting model...
make prediction...
fold#5: ndcg = 85659
make features...
fitting model...
make prediction...
fold#6: ndcg = 85489
make features...
fitting model...
make prediction...
fold#7: ndcg = 86035
make features...
fitting model...
make prediction...
fold#8: ndcg = 85668
make features...
fitting model...
make prediction...
fold#9: ndcg = 85639
make features...
fitting model...
make prediction...
fold#10: ndcg = 85552
averaged cv score: ndcg = 85737


In [95]:
counters_pipe.fit(np.hstack([train[col] for col in text_cols]))
train_data = np.hstack([counters_pipe.transform(train[col]) for col in text_cols])
test_data = np.hstack([counters_pipe.transform(test[col]) for col in text_cols])

lr.fit(train_data, train['target'])

prediction = pd.DataFrame()
prediction['context_id'] = test[0]
prediction['reply_id'] = test[4]
prediction['rank'] = - lr.predict(test_data)
prediction = prediction.sort_values(by=['context_id', 'rank'])[['context_id', 'reply_id']]

In [96]:
prediction.to_csv(path+'/data/sub_1.tsv',header=None, index=False, sep=' ')

In [236]:
train.to_csv(path+'/data/train_modified.tsv', sep=' ')

In [241]:
train = pd.read_csv(path+'/data/train_modified.tsv', sep=' ', index_col=0)

In [242]:
train.head()

,0,1,2,3,4,5,6,7,rank,target
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",0,не могу .,good,0.875352,3,2.626055
1,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",1,"нет , звонить буду я .",neutral,0.900968,2,1.801936
2,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",2,"слушай , я не мог уйти .",bad,0.884320,1,0.884320
3,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",3,я не прекращу звонить .,good,0.982530,3,2.947591
4,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",4,я звоню им .,good,0.838054,3,2.514161


In [112]:
import re
#https://github.com/ppgmg/github_public/blob/master/spell/symspell_python.py
max_edit_distance = 3 
verbose = 0
# 0: top suggestion
# 1: all suggestions of smallest edit distance
# 2: all suggestions <= max_edit_distance (slower, no early termination)

dictionary = {}
longest_word_length = 0

def get_deletes_list(w):
    '''given a word, derive strings with up to max_edit_distance characters
       deleted'''
    deletes = []
    queue = [w]
    for d in range(max_edit_distance):
        temp_queue = []
        for word in queue:
            if len(word)>1:
                for c in range(len(word)):  # character index
                    word_minus_c = word[:c] + word[c+1:]
                    if word_minus_c not in deletes:
                        deletes.append(word_minus_c)
                    if word_minus_c not in temp_queue:
                        temp_queue.append(word_minus_c)
        queue = temp_queue
        
    return deletes

def create_dictionary_entry(w):
    '''add word and its derived deletions to dictionary'''
    # check if word is already in dictionary
    # dictionary entries are in the form: (list of suggested corrections,
    # frequency of word in corpus)
    global longest_word_length
    new_real_word_added = False
    if w in dictionary:
        # increment count of word in corpus
        dictionary[w] = (dictionary[w][0], dictionary[w][1] + 1)  
    else:
        dictionary[w] = ([], 1)  
        longest_word_length = max(longest_word_length, len(w))
        
    if dictionary[w][1]==1:
        # first appearance of word in corpus
        # n.b. word may already be in dictionary as a derived word 
        # (deleting character from a real word)
        # but counter of frequency of word in corpus is not incremented 
        # in those cases)
        new_real_word_added = True
        deletes = get_deletes_list(w)
        for item in deletes:
            if item in dictionary:
                # add (correct) word to delete's suggested correction list 
                dictionary[item][0].append(w)
            else:
                # note frequency of word in corpus is not incremented
                dictionary[item] = ([w], 0)  
        
    return new_real_word_added

def create_dictionary(fname):

    total_word_count = 0
    unique_word_count = 0
    
    with open(fname) as file:
        for line in file:
            # separate by words by non-alphabetical characters      
            words = re.findall('[a-z]+', line.lower())  
            for word in words:
                total_word_count += 1
                if create_dictionary_entry(word):
                    unique_word_count += 1
        
    return dictionary

def dameraulevenshtein(seq1, seq2):
    """Calculate the Damerau-Levenshtein distance between sequences.
    This method has not been modified from the original.
    Source: http://mwh.geek.nz/2009/04/26/python-damerau-levenshtein-distance/
    
    This distance is the number of additions, deletions, substitutions,
    and transpositions needed to transform the first sequence into the
    second. Although generally used with strings, any sequences of
    comparable objects will work.
    Transpositions are exchanges of *consecutive* characters; all other
    operations are self-explanatory.
    This implementation is O(N*M) time and O(M) space, for N and M the
    lengths of the two sequences.
    >>> dameraulevenshtein('ba', 'abc')
    2
    >>> dameraulevenshtein('fee', 'deed')
    2
    It works with arbitrary sequences too:
    >>> dameraulevenshtein('abcd', ['b', 'a', 'c', 'd', 'e'])
    2
    """
    # codesnippet:D0DE4716-B6E6-4161-9219-2903BF8F547F
    # Conceptually, this is based on a len(seq1) + 1 * len(seq2) + 1 matrix.
    # However, only the current and two previous rows are needed at once,
    # so we only store those.
    oneago = None
    thisrow = range(1, len(seq2) + 1) + [0]
    for x in range(len(seq1)):
        # Python lists wrap around for negative indices, so put the
        # leftmost column at the *end* of the list. This matches with
        # the zero-indexed strings and saves extra calculation.
        twoago, oneago, thisrow = oneago, thisrow, [0] * len(seq2) + [x + 1]
        for y in range(len(seq2)):
            delcost = oneago[y] + 1
            addcost = thisrow[y - 1] + 1
            subcost = oneago[y - 1] + (seq1[x] != seq2[y])
            thisrow[y] = min(delcost, addcost, subcost)
            # This block deals with transpositions
            if (x > 0 and y > 0 and seq1[x] == seq2[y - 1]
                and seq1[x-1] == seq2[y] and seq1[x] != seq2[y]):
                thisrow[y] = min(thisrow[y], twoago[y - 2] + 1)
    return thisrow[len(seq2) - 1]

def get_suggestions(string, silent=False):
    '''return list of suggested corrections for potentially incorrectly
       spelled word'''
    if (len(string) - longest_word_length) > max_edit_distance:
        return []
    
    global verbose
    suggest_dict = {}
    min_suggest_len = float('inf')
    
    queue = [string]
    q_dictionary = {}  # items other than string that we've checked
    
    while len(queue)>0:
        q_item = queue[0]  # pop
        queue = queue[1:]
        
        # early exit
        if ((verbose<2) and (len(suggest_dict)>0) and 
              ((len(string)-len(q_item))>min_suggest_len)):
            break
        
        # process queue item
        if (q_item in dictionary) and (q_item not in suggest_dict):
            if (dictionary[q_item][1]>0):
            # word is in dictionary, and is a word from the corpus, and 
            # not already in suggestion list so add to suggestion 
            # dictionary, indexed by the word with value (frequency in
            # corpus, edit distance)
            # note q_items that are not the input string are shorter 
            # than input string since only deletes are added (unless 
            # manual dictionary corrections are added)
                assert len(string)>=len(q_item)
                suggest_dict[q_item] = (dictionary[q_item][1], 
                                        len(string) - len(q_item))
                # early exit
                if ((verbose<2) and (len(string)==len(q_item))):
                    break
                elif (len(string) - len(q_item)) < min_suggest_len:
                    min_suggest_len = len(string) - len(q_item)
            
            # the suggested corrections for q_item as stored in 
            # dictionary (whether or not q_item itself is a valid word 
            # or merely a delete) can be valid corrections
            for sc_item in dictionary[q_item][0]:
                if (sc_item not in suggest_dict):
                    
                    # compute edit distance
                    # suggested items should always be longer 
                    # (unless manual corrections are added)
                    assert len(sc_item)>len(q_item)

                    # q_items that are not input should be shorter 
                    # than original string 
                    # (unless manual corrections added)
                    assert len(q_item)<=len(string)

                    if len(q_item)==len(string):
                        assert q_item==string
                        item_dist = len(sc_item) - len(q_item)

                    # item in suggestions list should not be the same as 
                    # the string itself
                    assert sc_item!=string

                    # calculate edit distance using, for example, 
                    # Damerau-Levenshtein distance
                    item_dist = dameraulevenshtein(sc_item, string)
                    
                    # do not add words with greater edit distance if 
                    # verbose setting not on
                    if ((verbose<2) and (item_dist>min_suggest_len)):
                        pass
                    elif item_dist<=max_edit_distance:
                        assert sc_item in dictionary  # should already be in dictionary if in suggestion list
                        suggest_dict[sc_item] = (dictionary[sc_item][1], item_dist)
                        if item_dist < min_suggest_len:
                            min_suggest_len = item_dist
                    
                    # depending on order words are processed, some words 
                    # with different edit distances may be entered into
                    # suggestions; trim suggestion dictionary if verbose
                    # setting not on
                    if verbose<2:
                        suggest_dict = {k:v for k, v in suggest_dict.items() if v[1]<=min_suggest_len}
                
        # now generate deletes (e.g. a substring of string or of a delete)
        # from the queue item
        # as additional items to check -- add to end of queue
        assert len(string)>=len(q_item)
                    
        # do not add words with greater edit distance if verbose setting 
        # is not on
        if ((verbose<2) and ((len(string)-len(q_item))>min_suggest_len)):
            pass
        elif (len(string)-len(q_item))<max_edit_distance and len(q_item)>1:
            for c in range(len(q_item)): # character index        
                word_minus_c = q_item[:c] + q_item[c+1:]
                if word_minus_c not in q_dictionary:
                    queue.append(word_minus_c)
                    q_dictionary[word_minus_c] = None  # arbitrary value, just to identify we checked this
                     
    # queue is now empty: convert suggestions in dictionary to 
    # list for output
    
    # output option 1
    # sort results by ascending order of edit distance and descending 
    # order of frequency
    #     and return list of suggested word corrections only:
    # return sorted(suggest_dict, key = lambda x: 
    #               (suggest_dict[x][1], -suggest_dict[x][0]))

    # output option 2
    # return list of suggestions with (correction, 
    #                                  (frequency in corpus, edit distance)):
    as_list = suggest_dict.items()
    outlist = sorted(as_list, key=lambda term, freq, dist: (dist, -freq))
    
    if verbose==0:
        return outlist[0]
    else:
        return outlist

    '''
    Option 1:
    ['file', 'five', 'fire', 'fine', ...]
    
    Option 2:
    [('file', (5, 0)),
     ('five', (67, 1)),
     ('fire', (54, 1)),
     ('fine', (17, 1))...]  
    '''

def best_word(s, silent=False):
    try:
        return get_suggestions(s, silent)[0]
    except:
        return None

# ## main

# import time

# if __name__ == "__main__":
    
#     print "Please wait..."
#     time.sleep(2)
#     start_time = time.time()
#     try:
#         create_dictionary("big.txt")
#     except:
#         create_dictionary("testdata/big.txt")
#     run_time = time.time() - start_time
#     print '-----'
#     print '%.2f seconds to run' % run_time
#     print '-----'

#     print " "
#     print "Word correction"
#     print "---------------"
    
#     while True:
#         word_in = raw_input('Enter your input (or enter to exit): ')
#         if len(word_in)==0:
#             print "goodbye"
#             break
#         start_time = time.time()
#         print get_suggestions(word_in)
#         run_time = time.time() - start_time
#         print '-----'
#         print '%.5f seconds to run' % run_time
#         print '-----'
# print " "

In [115]:
vocab = pd.read_csv('/home/gas/Documents/Yandex_ML_track_2018/models/1grams-3.txt', sep='\t', header=None,
                   names=['freq', 'word'])
vocab = vocab.dropna().reset_index(drop=True)
vocab['len'] = vocab['word'].apply(lambda row: len(row))
vocab['first_symbol'] = vocab['word'].apply(lambda row: row[0])
vocab['last_symbol'] = vocab['word'].apply(lambda row: row[-1])

In [116]:
vocab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1054208 entries, 0 to 1054207
Data columns (total 5 columns):
freq            1054208 non-null int64
word            1054208 non-null object
len             1054208 non-null int64
first_symbol    1054208 non-null object
last_symbol     1054208 non-null object
dtypes: int64(2), object(3)
memory usage: 40.2+ MB


In [117]:
vocab.shape

(1054208, 5)

In [119]:
vocab.head()

,freq,word,len,first_symbol,last_symbol
0,6829968,и,1,и,и
1,5190215,в,1,в,в
2,3164900,не,2,н,е
3,2729210,на,2,н,а
4,2064408,с,1,с,с


In [120]:
def dameraulevenshtein(seq1, data, trsh):
    data = pd.concat([data[(data['first_symbol'] == seq1[0])&(data['last_symbol'] != seq1[-1])], 
                      data[(data['first_symbol'] != seq1[0])&(data['last_symbol'] == seq1[-1])],
                      data[(data['first_symbol'] != seq1[0])&(data['last_symbol'] != seq1[-1])]])
    #data = data[data['first_symbol'] == seq1[0]].append(data[data['first_symbol'] != seq1[0]])
    variants, closest = [], (np.inf, None)
    for i in data['word'].index:
        seq2 = data['word'].loc[i]
        oneago = None
        thisrow = list(range(1, len(seq2) + 1)) + [0]
        for x in range(len(seq1)):
            # Python lists wrap around for negative indices, so put the
            # leftmost column at the *end* of the list. This matches with
            # the zero-indexed strings and saves extra calculation.
            twoago, oneago, thisrow = oneago, thisrow, [0] * len(seq2) + [x + 1]
            for y in range(len(seq2)):
                delcost = oneago[y] + 1
                addcost = thisrow[y - 1] + 1
                subcost = oneago[y - 1] + (seq1[x] != seq2[y])
                thisrow[y] = min(delcost, addcost, subcost)
                # This block deals with transpositions
                if (x > 0 and y > 0 and seq1[x] == seq2[y - 1]
                    and seq1[x-1] == seq2[y] and seq1[x] != seq2[y]):
                    thisrow[y] = min(thisrow[y], twoago[y - 2] + 1)
        result = thisrow[len(seq2) - 1]
        if result < closest[0]:
            closest = (result, i)
        if result < trsh:
            variants.append(i)
        if len(variants) > 1:
            return data[data['freq'] == data['freq'].loc[variants].max()]['word'].values[0]
    if closest[0] <= trsh:
        return data['word'].loc[closest[1]]

In [9]:
%%time

query = 'сосатб'
if query not in vocab['word']:
    result = vocab[(vocab['len'] >= len(query)-3) & (vocab['len'] <= len(query)+3) & (vocab['len'] <= len(query)+3)]['word'].map(lambda x: dameraulevenshtein(query, x))
    result = vocab.loc[result[(result == result.min()) & (result.min() < 3)].index]['word'].values[0]

CPU times: user 24.3 s, sys: 16.3 ms, total: 24.4 s
Wall time: 24.4 s


In [123]:
%%time

query = 'сосатб'
trsh = 3
if query not in vocab['word'].values:
    result = dameraulevenshtein(query, vocab[(vocab['len'] >= len(query)-trsh) & (vocab['len'] <= len(query)+trsh) & (vocab['len'] <= len(query)+3)], trsh)
else:
    result = query

CPU times: user 915 ms, sys: 12 ms, total: 927 ms
Wall time: 926 ms


In [124]:
result

'состав'

In [246]:
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

In [ ]:
results = {
    'counter_char_ridge':[85700, 84100],
    'counter_word_ridge':[85738, 84248],
    'tfidf_word_ridge':[85774, 82714],
    'counter_svd_ridge':,
    'counter_tfidf_svd_ridge':,
    'counter_tfidf_svd_lgbm':[85743,8515],
    'w2v_tfidf_ridge':[85700, 82591],
    'w2v_tfidf_lgbm':[85541, 82858],
}